In [ ]:
"D:\data"

In [52]:
import os
import glob

# Function to count the number of images in a class folder
def count_images_in_class(class_folder):
    num_images = sum(len(files) for _, _, files in os.walk(class_folder))
    return num_images

# Assuming each class folder contains subfolders with images
class_folders = glob.glob('D:/data2/*')

# Print the number of images in each class folder
for class_folder in class_folders:
    print("Number of Images in", class_folder, ":", count_images_in_class(class_folder))



Number of Images in D:/data2\Bombarder : 2001
Number of Images in D:/data2\Civil : 1402
Number of Images in D:/data2\Divers : 119
Number of Images in D:/data2\Fighter : 1763
Number of Images in D:/data2\Recognition : 362
Number of Images in D:/data2\Transport : 3313


In [89]:
import os
import shutil
import random
import glob

# Function to count the number of images in a class folder
def count_images_in_class(class_folder):
    num_images = sum(len(files) for _, _, files in os.walk(class_folder))
    return num_images

# Function to collect all images within a class folder
def collect_images_in_class(class_folder):
    class_images = []
    for subdir, _, files in os.walk(class_folder):
        for file in files:
            class_images.append(os.path.join(subdir, file))
    return class_images

# Function to oversample a class folder to achieve the target number of images
def oversample_class_folder(class_folder, target_num_images):
    class_images = collect_images_in_class(class_folder)
    num_images = len(class_images)
    
    # If there are not enough images, oversample with replacement
    if num_images > 0 and target_num_images > 0:
        num_to_sample = target_num_images - num_images
        if num_to_sample > 0:
            oversampled_images = random.choices(class_images, k=num_to_sample)
            for img in oversampled_images:
                dst_path = os.path.splitext(img)[0] + '2' + os.path.splitext(img)[1]
                shutil.copy(img, dst_path)



class_folders = glob.glob('D:/data2/*')

# Calculate the maximum number of images across all class folders
max_num_images = max(count_images_in_class(class_folder) for class_folder in class_folders)

# Oversample each class folder to have the same number of images as the class with the most images
for class_folder in class_folders:
    oversample_class_folder(class_folder, max_num_images)

# Print the number of images in each class folder after oversampling
for class_folder in class_folders:
    print("Number of Images in", class_folder, ":", count_images_in_class(class_folder))


Number of Images in D:/data2\Bombarder : 3313
Number of Images in D:/data2\Civil : 3313
Number of Images in D:/data2\Divers : 3313
Number of Images in D:/data2\Fighter : 3313
Number of Images in D:/data2\Recognition : 3313
Number of Images in D:/data2\Transport : 3313


In [91]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to flatten the dataset structure and rename images
def flatten_and_rename(dataset_root):
    unique_id = 0
    for class_folder in os.listdir(dataset_root):
        class_path = os.path.join(dataset_root, class_folder)
        if os.path.isdir(class_path):
            for subfolder in os.listdir(class_path):
                subfolder_path = os.path.join(class_path, subfolder)
                if os.path.isdir(subfolder_path):
                    for file in os.listdir(subfolder_path):
                        src = os.path.join(subfolder_path, file)
                        dst = os.path.join(class_path, f"{unique_id}_{file}")
                        shutil.move(src, dst)
                        unique_id += 1
                    os.rmdir(subfolder_path)
                    
                    
# Define paths
dataset_root = 'D:/data2'

# Flatten the dataset structure and rename images
flatten_and_rename(dataset_root)

In [92]:
import os
import glob

# Function to count the number of images in a class folder
def count_images_in_class(class_folder):
    num_images = sum(len(files) for _, _, files in os.walk(class_folder))
    return num_images

# Assuming each class folder contains subfolders with images
class_folders = glob.glob('D:/data2/*')

# Print the number of images in each class folder
for class_folder in class_folders:
    print("Number of Images in", class_folder, ":", count_images_in_class(class_folder))

Number of Images in D:/data2\Bombarder : 3313
Number of Images in D:/data2\Civil : 3313
Number of Images in D:/data2\Divers : 3313
Number of Images in D:/data2\Fighter : 3313
Number of Images in D:/data2\Recognition : 3313
Number of Images in D:/data2\Transport : 3313


In [102]:
import tensorflow as tf

# Define parameters
batch_size = 38
image_size = (128, 128)



train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_root,
    seed=123,
    validation_split=0.2,
    subset='training',
    batch_size=batch_size,
    image_size=image_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_root,
    seed=123,
    validation_split=0.2,
    subset='validation',
    batch_size=batch_size,
    image_size=image_size
)


Found 19878 files belonging to 6 classes.
Using 15903 files for training.
Found 19878 files belonging to 6 classes.
Using 3975 files for validation.


In [103]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [104]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape based on your image size
input_shape = (128, 128, 3)  # Assuming RGB images

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Add dropout for regularization
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')  # Assuming 5 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     2,359,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,781,638 (10.61 MB)

 Trainable params: 2,781,638 (10.61 MB)

 Non-trainable params: 0 (0.00 B)

In [105]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 160s 372ms/step - accuracy: 0.3882 - loss: 3.7025 - val_accuracy: 0.7464 - val_loss: 0.6722
Epoch 2/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 141s 335ms/step - accuracy: 0.7389 - loss: 0.6774 - val_accuracy: 0.8521 - val_loss: 0.4162
Epoch 3/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 131s 312ms/step - accuracy: 0.8413 - loss: 0.4337 - val_accuracy: 0.8926 - val_loss: 0.2914
Epoch 4/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 136s 324ms/step - accuracy: 0.8818 - loss: 0.3182 - val_accuracy: 0.8999 - val_loss: 0.2719
Epoch 5/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 133s 318ms/step - accuracy: 0.9056 - loss: 0.2626 - val_accuracy: 0.9326 - val_loss: 0.1977


In [106]:
history = model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
419/419 ━━━━━━━━━━━━━━━━━━━━ 131s 312ms/step - accuracy: 0.9220 - loss: 0.2309 - val_accuracy: 0.9419 - val_loss: 0.1775
Epoch 2/2
419/419 ━━━━━━━━━━━━━━━━━━━━ 131s 313ms/step - accuracy: 0.9284 - loss: 0.2050 - val_accuracy: 0.9452 - val_loss: 0.1737


In [107]:
history = model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
419/419 ━━━━━━━━━━━━━━━━━━━━ 134s 320ms/step - accuracy: 0.9320 - loss: 0.1904 - val_accuracy: 0.9562 - val_loss: 0.1354
Epoch 2/2
419/419 ━━━━━━━━━━━━━━━━━━━━ 134s 319ms/step - accuracy: 0.9430 - loss: 0.1729 - val_accuracy: 0.9507 - val_loss: 0.1511


In [108]:
val_loss, val_accuracy = model.evaluate(val_ds)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_accuracy)

105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.9555 - loss: 0.1447
Validation loss: 0.15113408863544464
Validation accuracy: 0.950691819190979


In [110]:
model.save("C:/Users/peanu/OneDrive/Desktop/ENSA/ML/model_data2.keras")